In [15]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [16]:
N_train = 5000
N_bound = 100

np.random.seed(123)
train_x_y_t = lhs(3,N_train)
np.random.seed(12345)
bound_x_y = lhs(2,N_bound)
bound_x_y_t = np.concatenate((bound_x_y,np.zeros((N_bound,1))),1)

In [17]:
train_x_y_t = torch.from_numpy(train_x_y_t).float()
bound_x_y = torch.from_numpy(bound_x_y).float()
bound_x_y_t = torch.from_numpy(bound_x_y_t).float()

x = train_x_y_t[:,0:1]
y = train_x_y_t[:,1:2]
t = train_x_y_t[:,2:3]
x.requires_grad_()
y.requires_grad_()
t.requires_grad_()

tensor([[0.3976],
        [0.7691],
        [0.9756],
        ...,
        [0.2023],
        [0.6423],
        [0.6873]], requires_grad=True)

In [18]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

In [19]:
import scipy.integrate
from numpy import exp
from math import sqrt
import math
def InIn(x_y_t):
    x1 = x_y_t[:,0:1]
    x2 = x_y_t[:,1:2]
    t = x_y_t[:,2:3]
    P = np.zeros_like(t)
    for i in range(len(t)):
        f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t[i]))*(y1-y2)
        p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
        P[i] = p/(4*np.pi*t[i])
    return P

In [20]:
np.random.seed(5678)
test_x_y_t = lhs(3,500)
x1 = test_x_y_t[:,0:1]
x2 = test_x_y_t[:,1:2]
t_ = test_x_y_t[:,2:3]
test_u = np.zeros_like(t_)
for i in range(len(t_)):
    f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t_[i]))*(y1-y2)
    p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
    test_u[i] = p/(4*np.pi*t_[i])


In [21]:
test_x_y_t = torch.from_numpy(test_x_y_t).float()

### Labeled Data

In [22]:
np.random.seed(234)
gan_data_x_t = lhs(3,10)
gan_data_u = InIn(gan_data_x_t)

In [23]:
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

# Method5 
$\text { GA - PINN* }$
<div class="alert alert-info">`loss_function：`<br>
`Pre_training：`<br>
$
\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T
$
<br>
`Follow-up training：`<br>
$
\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T$

</div>

In [24]:
k=2

In [25]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs5 = NN_H2(3, 100, 4, 1)
PINNs5.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs5.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)
discriminator.cuda()

loss_all_5 = []
test_loss_5 = []

#########gpu############
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
PINNs5.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6 and it<1000:
    PINNs5.cuda()
    
    ##### loss_Bi  ######
    u_bound = PINNs5(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
    
    ##### loss f  ######
    
    u_inside = PINNs5(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    loss = loss_bound+loss_f
    pre_H = PINNs5(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    loss1_value = loss.item()
    
    loss_all_5.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs5.cpu()
    test_loss =  relative_l2(PINNs5(test_x_y_t).detach().numpy(),test_u)
    test_loss_5.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

It: 0 Loss: 0.32263725996017456
It: 100 Loss: 0.001767130452208221
It: 200 Loss: 0.0006751884357072413
It: 300 Loss: 0.000374616589397192
It: 400 Loss: 0.000188114820048213
It: 500 Loss: 0.0013421529438346624
It: 600 Loss: 0.0001307471829932183
It: 700 Loss: 9.013738599605858e-05
It: 800 Loss: 6.806811870774254e-05
It: 900 Loss: 5.379993672249839e-05


0.0001633025676710531

In [26]:
while  loss1_value>5e-6:
    PINNs5.cuda()
    
    ##############loss D############
    pre_H = PINNs5(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    

    
    ##############loss trick############
    pre_H = PINNs5(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  
    

    ##### loss_Bi  ######
    u_bound = PINNs5(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    
    
    ##### loss f  ######
    
    u_inside = PINNs5(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    

    loss = loss_bound+loss_f
    pre_H = PINNs5(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    
    loss1_value = loss.item()
    
    
    loss_all_5.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    
    #########  test_loss NRMSE  #########
    PINNs5.cpu()
    test_loss =  relative_l2(PINNs5(test_x_y_t).detach().numpy(),test_u)
    test_loss_5.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

It: 1000 Loss: 0.0002247704833280295
It: 1100 Loss: 0.00022318089031614363
It: 1200 Loss: 0.00011627576895989478
It: 1300 Loss: 0.002363355830311775
It: 1400 Loss: 0.00015286503185052425
It: 1500 Loss: 7.906803512014449e-05
It: 1600 Loss: 0.00027986799250356853
It: 1700 Loss: 8.90104056452401e-05
It: 1800 Loss: 0.0002302155044162646
It: 1900 Loss: 5.53602039872203e-05
It: 2000 Loss: 9.477922867517918e-05
It: 2100 Loss: 3.107157681370154e-05
It: 2200 Loss: 3.25686160067562e-05
It: 2300 Loss: 5.387669261835981e-06


4.963426363246981e-06

# Results

## Save data

In [2]:
# np.save('../experimental_data/method_5/test_loss_5',test_loss_5)
# np.save('../experimental_data/method_5/loss_all_5',loss_all_5)
# torch.save(PINNs5,'../saved_model/PINNs5')

## Epoch and NRMSE

In [3]:
Epochs = len(test_loss_5)
NRMSE = relative_l2(PINNs5(test_x_y_t).detach().numpy(),test_u)

print('Epochs:',Epochs,'NRMSE:',NRMSE)

NameError: name 'test_loss_5' is not defined